In [1]:
from pymatgen.io.vasp import Vasprun
from pymatgen.electronic_structure.plotter import BSPlotterProjected

vr = Vasprun("vasprun.xml", parse_projected_eigen=True)
bs = vr.get_band_structure(kpoints_filename="KPOINTS", line_mode=True, efermi=vr.efermi)

plotter = BSPlotterProjected(bs)
# Map each element to itself → element-projected fat bands
plot = plotter.get_projected_plot({"Cu":["Cu"], "Ag":["Ag"], "P":["P"], "S":["S"]})
plot.show()  # or: plot.savefig("bands_elem_proj.png", dpi=300, bbox_inches="tight")

AttributeError: 'BSPlotterProjected' object has no attribute 'get_projected_plot'

In [14]:
# --- Ag/S element-projected band structure (Jupyter version) ---
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.io.vasp import Poscar, Vasprun, Procar
from pymatgen.electronic_structure.core import Spin
from pymatgen.electronic_structure.bandstructure import BandStructureSymmLine

ELEMENTS = ["Ag", "S"]

def load_bs(vasprun_path, kpoints_path):
    vr = Vasprun(vasprun_path, parse_projected_eigen=True, occu_tol=1e-8, parse_potcar_file=False)
    bs = vr.get_band_structure(kpoints_filename=kpoints_path, line_mode=True, efermi=vr.efermi)
    return vr, bs

def kaxis_and_ticks(bs: BandStructureSymmLine):
    kdist = np.array(bs.distance)
    ticks = bs.get_ticks()
    return np.array(kdist), ticks["distance"], [lbl.replace("|","│") for lbl in ticks["label"]]

def proj_from_bs(bs, elements):
    if not bs.projections:
        return None
    pd = bs.get_projection_on_elements(elements)
    out = {el:{} for el in elements}
    for el in elements:
        for sp, arr in pd[el].items():
            out[el][sp] = np.array(arr)
    return out

def proj_from_procar(procar_path, poscar_path, elements):
    procar = Procar(procar_path)
    species = [str(s.specie) for s in Poscar.from_file(poscar_path).structure]
    elem_to_ions = {el:[i for i,sp in enumerate(species) if sp==el] for el in elements}
    spins = [Spin.up] if not procar.is_spin else [Spin.up,Spin.down]
    nk, nb = procar.nkpoints, procar.nbands
    elem_proj = {el:{sp:np.zeros((nb,nk)) for sp in spins} for el in elements}
    for si, sp in enumerate(spins):
        for k in range(nk):
            for b in range(nb):
                for el in elements:
                    val = sum(procar.data[si][k][b]['ion'][ion]['tot'] for ion in elem_to_ions[el])
                    elem_proj[el][sp][b,k]=val
    total=np.zeros((nb,nk))
    for sp in spins: total+=sum(elem_proj[el][sp] for el in elements)
    total[total==0]=1
    for el in elements:
        for sp in spins: elem_proj[el][sp]/=total
    return elem_proj

def plot_elem_projected(bs, elem_proj, elements, emin=-3, emax=3, size=40):
    kdist, xticks, xlabels = kaxis_and_ticks(bs)
    efermi = bs.efermi
    spins = [Spin.up] if not bs.is_spin_polarized else [Spin.up, Spin.down]
    colors = {"Ag":"#1f77b4", "S":"#d62728"}
    fig, ax = plt.subplots(figsize=(7,5), dpi=150)
    # gray reference bands
    for sp in spins:
        for band in bs.bands[sp]:
            ax.plot(kdist, band-efermi, lw=0.7, color="#888888", zorder=1)
    # element weights
    for el in elements:
        for sp in spins:
            bands=bs.bands[sp]; w=elem_proj[el][sp]
            nb,nk=bands.shape
            x=np.tile(kdist,nb); y=(bands-efermi).reshape(-1)
            weight=w.reshape(-1)
            s=size*(weight/weight.max() if weight.max()>0 else weight)
            ax.scatter(x,y,s=s,c=colors[el],alpha=0.85,edgecolors='none',label=el if sp==spins[0] else None,zorder=3)
    ax.set_xlim(kdist.min(),kdist.max()); ax.set_ylim(emin,emax)
    ax.set_ylabel("Energy (eV) relative to $E_F$")
    ax.set_xticks(xticks); ax.set_xticklabels(xlabels)
    for x in xticks: ax.axvline(x,color="k",lw=0.5,alpha=0.3)
    ax.axhline(0,color="k",lw=0.8,ls="--",alpha=0.6)
    ax.legend(frameon=False)
    plt.tight_layout(); plt.show()

# --- Run it here ---
vasprun_path = "vasprun.xml"
kpoints_path = "KPOINTS"
poscar_path  = "POSCAR"
procar_path  = "PROCAR"  # used only if vasprun lacks projections

vr, bs = load_bs(vasprun_path, kpoints_path)
elem_proj = proj_from_bs(ELEMENTS)
if elem_proj is None:
    print("No projections in vasprun.xml → using PROCAR")
    elem_proj = proj_from_procar(procar_path, poscar_path, ELEMENTS)

plot_elem_projected(bs, elem_proj, ELEMENTS, emin=-2.5, emax=2.5, size=40)


TypeError: proj_from_bs() missing 1 required positional argument: 'elements'